In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import gc

# ----------------------------------
# Load Flux Data
# ----------------------------------
X_flux = np.load("X_flux_aligned.npy")
print("✅ Loaded X_flux_aligned.npy:", X_flux.shape)  # (samples, 2000)

# Add channel dimension: (samples, 2000, 1)
if len(X_flux.shape) == 2:
    X_flux = X_flux[..., np.newaxis]

X_tensor = torch.tensor(X_flux, dtype=torch.float32)
dataset = TensorDataset(X_tensor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# ----------------------------------
# Positional Encoding
# ----------------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # (1, seq_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

# ----------------------------------
# Transformer Encoder Block
# ----------------------------------
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model=64, nhead=4, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                        dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    def forward(self, x):
        x = x.permute(1, 0, 2)  # (seq_len, batch, d_model)
        out = self.transformer_encoder(x)
        return out.permute(1, 0, 2)  # (batch, seq_len, d_model)

# ----------------------------------
# Optimized Autoencoder Model
# ----------------------------------
class FluxTransformerAutoencoder(nn.Module):
    def __init__(self, seq_len=2000, d_model=64):
        super().__init__()
        self.input_proj = nn.Linear(1, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len=seq_len)
        self.encoder = TransformerEncoderBlock(d_model=d_model)
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x_proj = self.input_proj(x)                    # (batch, seq_len, d_model)
        x_proj = self.positional_encoding(x_proj)      # add positional info
        encoded = self.encoder(x_proj)                 # (batch, seq_len, d_model)
        decoded = self.decoder(encoded)                # (batch, seq_len, 1)
        return decoded + x  # residual connection (only works if same shape)

# ----------------------------------
# Model Setup
# ----------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FluxTransformerAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ----------------------------------
# Training Loop
# ----------------------------------
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
        x = batch[0].to(device)
        output = model(x)
        loss = criterion(output, x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"✅ Epoch {epoch+1} Loss: {total_loss / len(loader):.6f}")

# ----------------------------------
# Embedding Extraction
# ----------------------------------
print("\n🔍 Extracting embeddings...")
model.eval()
model = model.cpu()

embedding_list = []
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Extracting embeddings"):
        x = batch[0]
        x_proj = model.input_proj(x)
        x_proj = model.positional_encoding(x_proj)
        encoded = model.encoder(x_proj)
        pooled = encoded.mean(dim=1).numpy()  # Global average pooling
        embedding_list.append(pooled)
        del x, x_proj, encoded, pooled
        gc.collect()

# Save final embeddings
flux_embeddings = np.concatenate(embedding_list, axis=0)
np.save("flux_embeddings.npy", flux_embeddings)
print("✅ flux_embeddings.npy saved:", flux_embeddings.shape)


c:\Users\Roshen Hasangha\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


✅ Loaded X_flux_aligned.npy: (1088, 2000)


Epoch 1/10: 100%|██████████| 68/68 [10:07<00:00,  8.94s/it]


✅ Epoch 1 Loss: 0.004037


Epoch 2/10: 100%|██████████| 68/68 [08:55<00:00,  7.87s/it]


...

✅ Epoch 9 Loss: 0.000004


Epoch 10/10: 100%|██████████| 68/68 [08:24<00:00,  7.42s/it]


✅ Epoch 10 Loss: 0.000003

🔍 Extracting embeddings...


Extracting embeddings: 100%|██████████| 68/68 [00:54<00:00,  1.25it/s]

✅ flux_embeddings.npy saved: (1088, 64)


In [1]:
import numpy as np
data = np.load("X_flux_aligned.npy")
print(data.shape)

(1088, 2000)
